<a href="https://colab.research.google.com/github/viplazylmht/DstDedicated/blob/main/Setup_DST_Dedicated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From here we can found a fast way to start a dedicated server from google colaboratory (free).  
The maximum runtime of the server at here is about 12 hours (or less).  

If you want to save states of the game, please download your cluster' folders normally on the left panel (Files).

# Original tutorial post (by klei 4rum)

https://forums.kleientertainment.com/forums/topic/64441-dedicated-server-quick-setup-guide-linux/

Make sure this machine support x86 environment (32bit)

In [ ]:
!sudo dpkg --add-architecture i386
!sudo apt-get update

!sudo apt-get install libstdc++6:i386 libgcc1:i386 libcurl4-gnutls-dev:i386

## Connect to working dir (can be located on google drive)

In [52]:
PR_DIR="/content/"
%cd $PR_DIR

/content


## install steamcmd

In [53]:
!mkdir -p steamcmd
%cd steamcmd

/content/steamcmd


In [54]:
!wget "https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz"

!tar -xvzf steamcmd_linux.tar.gz
!rm steamcmd_linux.tar.gz

!chmod -R 0755 *

%cd $PR_DIR

--2021-06-28 09:06:05--  https://steamcdn-a.akamaihd.net/client/installer/steamcmd_linux.tar.gz
Resolving steamcdn-a.akamaihd.net (steamcdn-a.akamaihd.net)... 23.35.70.16, 23.35.70.67
Connecting to steamcdn-a.akamaihd.net (steamcdn-a.akamaihd.net)|23.35.70.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428561 (2.3M) [application/octet-stream]
Saving to: ‘steamcmd_linux.tar.gz’

steamcmd_linux.tar. 100%[===================>]   2.32M  --.-KB/s    in 0.07s   

2021-06-28 09:06:05 (33.0 MB/s) - ‘steamcmd_linux.tar.gz’ saved [2428561/2428561]

steamcmd.sh
linux32/steamcmd
linux32/steamerrorreporter
linux32/libstdc++.so.6
linux32/crashhandler.so
/content


## Mods

My mods file for my server is `https://drive.google.com/uc?id=1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG`, hence you can replace by your own.

In [69]:
%cd $PR_DIR
!mkdir -p dontstarvetogether_dedicated_server
%cd dontstarvetogether_dedicated_server/

!gdown --id 1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG

/content
/content/dontstarvetogether_dedicated_server
Downloading...
From: https://drive.google.com/uc?id=1wEqvUR9W24amYqJbCnfm4KfZGEgKo9fG
To: /content/dontstarvetogether_dedicated_server/mods.zip
297MB [00:03, 94.8MB/s]


Extract mods file to mods folder

In [70]:
!unzip -q -o mods.zip
!ls

bin		      data	      linux64  mods.zip   steamclient.so
cached_mod_manifests  dontstarve.xpm  mods     steamapps  version.txt


## Klei and server

In [57]:
%cd $PR_DIR 
!mkdir -p DoNotStarveTogether

/content


In [58]:
%cd $PR_DIR/DoNotStarveTogether

/content/DoNotStarveTogether


Download my own server. You can use this too becasue there is only a world seed (Start at Day1). This is a `shipwrecked extended` world create by [Tropical Experience | The Volcano Biome](https://steamcommunity.com/sharedfiles/filedetails/?id=1505270912) mod.

In [71]:
%cd $PR_DIR/DoNotStarveTogether

!wget -q https://cdn.discordapp.com/attachments/817270986423664670/859004097730445372/MyDediServer.zip

!unzip -q -o MyDediServer.zip
!rm MyDediServer.zip

!ls

/content/DoNotStarveTogether
MyDediServer


Please note that the name of dedicated server is `MyDediServer`, you need to rename again to make sure this match together.

After that, please use your own `SECRET TOKEN` (can be gen from [Klei Server page](https://accounts.klei.com/account/game/servers?game=DontStarveTogether)) instead of my token, then replace your token here:

In [60]:
YOUR_TOKEN_PLS = "PLEASE REPLACE ME INSIDE THE QUOTE"

assert YOUR_TOKEN_PLS != "PLEASE REPLACE ME INSIDE THE QUOTE"

Make sure the previnous cell run complete, then followup:

In [61]:
with open('MyDediServer/cluster_token.txt', 'wt') as f:
    f.write(YOUR_TOKEN_PLS)

!cat MyDediServer/cluster_token.txt

pds-g^KU_1_Xnbk8D^aPVPDuOXAQTNkD51xJc1CX5ytIWVfegX6W8a4Sgnf1Q=

Copy server files to `.klei` folders.

In [72]:
%cd ~

!mkdir -p .klei
!mkdir -p .klei/DoNotStarveTogether/

!cp -r $PR_DIR/DoNotStarveTogether/* ~/.klei/DoNotStarveTogether

!ls $PR_DIR/DoNotStarveTogether/

/root
MyDediServer


## RUN

Make auto script

In [63]:
%cd $PR_DIR
run_bash = """#!/bin/bash

DST_DIR=/content

steamcmd_dir="$DST_DIR/steamcmd"
install_dir="$DST_DIR/dontstarvetogether_dedicated_server"
cluster_name="MyDediServer"
dontstarve_dir="$DST_DIR/DoNotStarveTogether"

function fail()
{
	echo Error: "$@" >&2
	exit 1
}

function check_for_file()
{
	if [ ! -e "$1" ]; then
		fail "Missing file: $1"
	fi
}

cd "$steamcmd_dir" || fail "Missing $steamcmd_dir directory!"

check_for_file "steamcmd.sh"
check_for_file "$dontstarve_dir/$cluster_name/cluster.ini"
check_for_file "$dontstarve_dir/$cluster_name/cluster_token.txt"
check_for_file "$dontstarve_dir/$cluster_name/Master/server.ini"
check_for_file "$dontstarve_dir/$cluster_name/Caves/server.ini"

./steamcmd.sh +force_install_dir "$install_dir" +login anonymous +app_update 343050 validate +quit

check_for_file "$install_dir/bin"

cd "$install_dir/bin" || fail

run_shared=(./dontstarve_dedicated_server_nullrenderer)
run_shared+=(-console)
run_shared+=(-cluster "$cluster_name")
run_shared+=(-monitor_parent_process $$)

"${run_shared[@]}" -shard Caves  | sed 's/^/Caves:  /' &
"${run_shared[@]}" -shard Master | sed 's/^/Master: /'

"""

with open("run_dedicated_servers.sh", 'wt') as f:
    f.write(run_bash)

/content


Then just run this script, almost done.  

Please note that we need to download DST game (abount 2.3GB) when the first time run, and when you factory reset runtime. 

In [ ]:
%cd $PR_DIR
!chmod u+x run_dedicated_servers.sh
!./run_dedicated_servers.sh

In [65]:
# Master: [00:00:26]: World generated on build 469426 with save version: 5.065, using seed: 1624866836
# Caves:  [00:00:32]: World generated on build 469426 with save version: 5.065, using seed: 1624866835	

If you want to stop this dedicated server, just go to the toolbar, find `Runtime` option, and click `Interrupt execute`.

## Backup save game

In [74]:
%cd ~/.klei/DoNotStarveTogether

/root/.klei/DoNotStarveTogether


In [ ]:
!zip -r ServerBackup.zip MyDediServer

Goo, download your ServerBackup.zip, Done